# Eigenvalue extraction tests in a single-phase RLC circuit in DP & EMT domains

In [ ]:
import dpsimpy
import math
import villas.dataprocessing.readtools as rt

In [ ]:
# function to assert that two complex numbers are close
def assert_complex_isclose(expected, actual, rel_tol=1e-6):
    assert math.isclose(expected.real, actual.real, rel_tol=rel_tol), "Real parts not close: {} vs {}".format(expected.real, actual.real)
    assert math.isclose(expected.imag, actual.imag, rel_tol=rel_tol), "Imaginary parts not close: {} vs {}".format(expected.imag, actual.imag)

In [ ]:
# Circuit parameters
v_volt = 8.5
r_ohm = 100
l_henry = 5
c_farad = 250e-6
deltaT = 1e-4
final_time = 1e-3

Analytical solution

In [ ]:
eigenvalue0_expected = ((c_farad**2*r_ohm**2 - 4*l_henry*c_farad)**(1/2) - c_farad*r_ohm)/(2*c_farad*l_henry)
eigenvalue1_expected = -((c_farad**2*r_ohm**2 - 4*l_henry*c_farad)**(1/2) + c_farad*r_ohm)/(2*c_farad*l_henry)

print('Expected eigenvalue 0: ' + str(eigenvalue0_expected))
print('Expected eigenvalue 1: ' + str(eigenvalue1_expected))

Test 1: Eigenvalues extracted from simulation in DP domain match analytical solution.

In [ ]:
# DPsim DP simulation
name = 'DP_SinglePhaseRLC_Test'
log_dir = "logs/" + name
dpsimpy.Logger.set_log_dir(log_dir)

# Create nodes
gnd = dpsimpy.dp.SimNode.gnd
n1 = dpsimpy.dp.SimNode('n1')
n2 = dpsimpy.dp.SimNode('n2')
n3 = dpsimpy.dp.SimNode('n3')

# Create components
vs = dpsimpy.dp.ph1.VoltageSource('vs')
vs.set_parameters(V_ref=complex(v_volt,0))
r = dpsimpy.dp.ph1.Resistor('r')
r.set_parameters(R=r_ohm)
l = dpsimpy.dp.ph1.Inductor('l')
l.set_parameters(L=l_henry)
c = dpsimpy.dp.ph1.Capacitor('c')
c.set_parameters(C=c_farad)

# Set connections
vs.connect([gnd, n1])
l.connect([n1, n2])
c.connect([n2, n3])
r.connect([n3, gnd])

# Create topology
system = dpsimpy.SystemTopology(50, [n1, n2, n3], [vs, r, l, c])

# Configure and run simulation
sim = dpsimpy.Simulation(name)
sim.set_domain(dpsimpy.Domain.DP)
sim.set_system(system)
sim.do_eigenvalue_extraction(True)
sim.set_time_step(deltaT)
sim.set_final_time(final_time)
sim.run()

# Read log file
eigenvalues_timeseries = rt.read_timeseries_dpsim(log_dir + '/eigenvalues.csv')
eigenvalue0 = eigenvalues_timeseries['eigenvalues_0'].values
eigenvalue1 = eigenvalues_timeseries['eigenvalues_1'].values

# Assert
nSamples = int(final_time/deltaT)

assert len(eigenvalue0) == nSamples
assert len(eigenvalue1) == nSamples

for i in range(nSamples):
    assert_complex_isclose(eigenvalue0_expected, eigenvalue0[i], 1e-8)
    assert_complex_isclose(eigenvalue1_expected, eigenvalue1[i], 1e-8)

Test 2: Eigenvalues extracted from simulation in EMT domain match analytical solution.

In [ ]:
# DPsim EMT simulation
name = 'EMT_SinglePhaseRLC_Test'
log_dir = "logs/" + name
dpsimpy.Logger.set_log_dir(log_dir)

# Create nodes
gnd = dpsimpy.emt.SimNode.gnd
n1 = dpsimpy.emt.SimNode('n1')
n2 = dpsimpy.emt.SimNode('n2')
n3 = dpsimpy.emt.SimNode('n3')

# Create components
vs = dpsimpy.emt.ph1.VoltageSource('vs')
vs.set_parameters(V_ref=complex(v_volt,0))
r = dpsimpy.emt.ph1.Resistor('r')
r.set_parameters(R=r_ohm)
l = dpsimpy.emt.ph1.Inductor('l')
l.set_parameters(L=l_henry)
c = dpsimpy.emt.ph1.Capacitor('c')
c.set_parameters(C=c_farad)

# Set connections
vs.connect([gnd, n1])
l.connect([n1, n2])
c.connect([n2, n3])
r.connect([n3, gnd])

# Create topology
system = dpsimpy.SystemTopology(50, [n1, n2, n3], [vs, r, l, c])

# Configure and run simulation
sim = dpsimpy.Simulation(name)
sim.set_domain(dpsimpy.Domain.EMT)
sim.set_system(system)
sim.do_eigenvalue_extraction(True)
sim.set_time_step(deltaT)
sim.set_final_time(final_time)
sim.run()

# Read log file
eigenvalues_timeseries = rt.read_timeseries_dpsim(log_dir + '/eigenvalues.csv')
eigenvalue0 = eigenvalues_timeseries['eigenvalues_0'].values
eigenvalue1 = eigenvalues_timeseries['eigenvalues_1'].values

# Assert
nSamples = int(final_time/deltaT)

assert len(eigenvalue0) == nSamples
assert len(eigenvalue1) == nSamples

for i in range(nSamples):
    assert_complex_isclose(eigenvalue0_expected, eigenvalue0[i], 1e-8)
    assert_complex_isclose(eigenvalue1_expected, eigenvalue1[i], 1e-8)